In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('5fs_impedance_training_set_10_final.xlsx')
df

,ImP_exp_diff_mean,ImP_exp_dur_sd,ImP_RRV_RMSSD,ImP_insp_flow_sd,ImP_ie_ratio_sd,ImP_duty_cycle_mean,ImP_insp_amp_sd,ImP_exp_amp_sd,ImP_RSP_Symmetry_PeakTrough,ImP_ie_ratio_mean,Task_Label,Participant,Classification
0,0.118857,0.171499,382.434308,15663.415593,0.048638,0.489158,50985.092943,46774.304708,0.517348,0.958792,12.0a,10785,0
1,0.225143,0.173505,390.361005,20811.126431,0.069124,0.485452,30734.204708,31615.925634,0.527694,0.945842,12.0b,10785,0
2,0.236000,0.222699,434.669530,16277.918645,0.050025,0.476758,31615.118946,26435.238020,0.520548,0.912526,12.0c,10785,0
3,0.250667,0.190379,403.425334,15714.039237,0.071244,0.483507,29114.145601,25312.370962,0.530733,0.938632,12.0d,10785,0
4,0.140800,0.174310,483.103302,6162.134521,0.124387,0.481444,43005.468164,28574.347285,0.527149,0.936589,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,0.240000,0.274684,318.223192,3850.325373,0.265283,0.489451,8019.865362,14865.886766,0.523436,0.988002,85,98586,0
2570,0.708000,0.764264,1236.868088,72599.234334,0.375168,0.427189,121971.615000,102499.398673,0.511816,0.809380,87,98586,1
2571,0.119556,0.122849,159.285907,4549.907724,0.082769,0.499069,6323.609559,12526.615103,0.543171,0.999821,89,98586,0
2572,1.567500,0.980966,2899.130116,21555.712739,0.546540,0.482030,47869.716992,51080.627080,0.472444,1.070194,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.18857100e-01, 1.71499300e-01, 3.82434308e+02, ...,
         4.67743047e+04, 5.17347900e-01, 9.58791500e-01],
        [2.25142900e-01, 1.73505000e-01, 3.90361005e+02, ...,
         3.16159256e+04, 5.27694200e-01, 9.45842000e-01],
        [2.36000000e-01, 2.22698600e-01, 4.34669530e+02, ...,
         2.64352380e+04, 5.20547800e-01, 9.12525700e-01],
        ...,
        [1.19555600e-01, 1.22848900e-01, 1.59285907e+02, ...,
         1.25266151e+04, 5.43171200e-01, 9.99821100e-01],
        [1.56750000e+00, 9.80965500e-01, 2.89913012e+03, ...,
         5.10806271e+04, 4.72444400e-01, 1.07019450e+00],
        [2.16500000e-01, 1.61567600e-01, 3.19002015e+02, ...,
         1.69910575e+04, 5.20748900e-01, 9.41270500e-01]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['ImP_exp_diff_mean',
  'ImP_exp_dur_sd',
  'ImP_RRV_RMSSD',
  'ImP_insp_flow_sd',
  'ImP_ie_ratio_sd',
  'ImP_duty_cycle_mean',
  'ImP_insp_amp_sd',
  'ImP_exp_amp_sd',
  'ImP_RSP_Symmetry_PeakTr

In [4]:
#running the nested cross-validation for the Accelerometer method using Gradient Boosting
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : (1, 4, 20),
    'solver' : ['liblinear','lbfgs']
}


lrc = LogisticRegression(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=lrc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melisa/anaconda3/lib/python3.1

nested_accuracy_scores:    [0.92424242 0.87878788 0.8030303  0.89393939 0.87878788 0.92424242
 0.92424242 0.93939394 0.90909091 0.77272727 0.89393939 0.98484848
 0.90909091 0.90909091 0.95454545 0.86363636 0.87878788 0.87878788
 0.90909091 0.6969697  0.81818182 0.89393939 0.92424242 0.95454545
 0.92424242 0.78787879 0.93939394 0.90909091 0.87878788 0.81818182
 0.90909091 0.87878788 0.92424242 1.         0.93939394 0.74242424
 0.93939394 0.84848485 0.95454545]
mean score:            0.8901
nested_precision_scores:    [0.93230174 0.89818182 0.80595875 0.89962333 0.89818182 0.92611833
 0.92532151 0.94008264 0.91223776 0.78861306 0.91788856 0.98545455
 0.91223776 0.90909091 0.95757576 0.86694493 0.8885851  0.8885851
 0.92045455 0.71868687 0.82255245 0.90909091 0.92981093 0.95459141
 0.92532151 0.78512397 0.94213287 0.90909091 0.88234266 0.83787879
 0.91223776 0.88699495 0.92611833 1.         0.93939394 0.75887486
 0.94008264 0.86767677 0.95757576]
mean score:            0.8969
nested_recal

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.7917
Specificity: 1.0000
Fold 2:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 3:
Sensitivity: 0.5833
Specificity: 0.9286
Fold 4:
Sensitivity: 0.7500
Specificity: 0.9762
Fold 5:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8333
Specificity: 0.9762
Fold 7:
Sensitivity: 0.9167
Specificity: 0.9286
Fold 8:
Sensitivity: 0.8750
Specificity: 0.9762
Fold 9:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 10:
Sensitivity: 0.7917
Specificity: 0.7619
Fold 11:
Sensitivity: 1.0000
Specificity: 0.8333
Fold 12:
Sensitivity: 1.0000
Specificity: 0.9762
Fold 13:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 14:
Sensitivity: 0.8333
Specificity: 0.9524
Fold 15:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 16:
Sensitivity: 0.7083
Specificity: 0.9524
Fold 17:
Sensitivity: 0.9167
Specificity: 0.8571
Fold 18:
Sensitivity: 0.9167
Specificity: 0.8571
Fold 19:
Sensitivity: 0.7500
Specificity: 1.0000
Fold 20:
Sensitivity: 0.7083
Specificity: 0.6905
Fold 21:
Sensitivity: 0.7917


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8900543900543902, SD=0.0642559219590999, 95% CI=(0.8698879636014833, 0.9102208165072971), Min=0.696969696969697, Max=1.0
test_precision: Mean=0.8968978524147891, SD=0.06049640107753749, 95% CI=(0.8779113343507846, 0.9158843704787937), Min=0.7186868686868687, Max=1.0
test_recall: Mean=0.8900543900543902, SD=0.0642559219590999, 95% CI=(0.8698879636014833, 0.9102208165072971), Min=0.696969696969697, Max=1.0
test_f1_score: Mean=0.8805299191536536, SD=0.0678037707394258, 95% CI=(0.8592500166301673, 0.90180982167714), Min=0.6866096866096867, Max=1.0
test_roc_auc: Mean=0.9532458282458282, SD=0.054903333091551605, 95% CI=(0.9360146689214786, 0.9704769875701778), Min=0.7261904761904763, Max=1.0
sensitivity_scores: Mean=0.8386752136752137, SD=0.10609163773616119, 95% CI=(0.805378839668837, 0.8719715876815903), Min=0.5833333333333334, Max=1.0
specificity_scores: Mean=0.9194139194139194, SD=0.08131559836242101, 95% CI=(0.8938933917512134, 0.9449344470766253), Min=0.6904761904

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')


Fold 1 - Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 2 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 3 - Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 4 - Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 5 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 6 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 7 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 8 - Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 9 - Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 10 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 11 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 12 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 13 - Best hyperparameters: {'

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: C - Most frequent value: 20 (Count: 16)
Hyperparameter: penalty - Most frequent value: l1 (Count: 39)
Hyperparameter: solver - Most frequent value: liblinear (Count: 39)
